In [2]:
from bs4 import BeautifulSoup
from itertools import chain
from collections import Counter
from nltk.corpus import stopwords
import urllib
from lxml import html
import numpy as np
import requests
import pandas as pd
import nltk
from nltk import corpus
from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import requests_cache
requests_cache.install_cache('demo_cache')
import math
from matplotlib import pyplot
import statsmodels
from scipy.interpolate import interp1d
import warnings
from statsmodels.tsa.arima_process import arma_generate_sample
import statsmodels.api
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
warnings.filterwarnings("ignore")

In [106]:
Country = ["Los Angeles","San Diego","Orange","Riverside","Santa Clara","Sacramento","Fresno","San Francisco","Santa Barbara","Santa Cruz","Yolo","San Mateo","San Joaquin","Kern"]
url_wi = "https://en.wikipedia.org/wiki/List_of_California_locations_by_income"
page = requests.get(url_wi)
soup = BeautifulSoup(page.content, 'html.parser')
dat = soup.find_all("table",attrs={"class": "wikitable sortable"})
dat_1 = dat[0].find_all("td")
re_county = []
re_name = []
for i in dat_1:
    if i.a==None:
        re_county.append(str(i.text))
    else:
        re_name.append(i.text)
re_county = np.asarray(re_county)
re_county = np.split(re_county, 58)
data = pd.DataFrame(re_county)
data["County"] = re_name
del data[2]
del data[3]
data.columns = ['population', 'population density','median family income','County']

In [31]:
de = []
for i in range(0,len(data["County"])):
    if data["County"][i] not in Country:
        de.append(i)
data = data.drop(data.index[de])

In [16]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [42]:
lat = []
lon = []
get_name = [str(i) for i in data['County']]
for i in get_name:
    location = geolocator.geocode(i)
    lat.append(location.latitude)
    lon.append(location.longitude)

In [45]:
income = []
for i in list(data['median family income']):
    i = i.replace("$","")
    i = i.replace(",","")
    i = int(i)
    income.append(i)
rate_in = [(float(i)/float(max(income)))*100 for i in income ]
str_rate_in = [str(i) for i in rate_in]
str_income = [str(i) for i in income ]

In [46]:
pd = []
for i in list(data['population density']):
    i = i.replace(",","")
    i = float(i)
    pd.append(i)
rate_pd = [(float(i)/float(max(pd)))*100 for i in pd ]
str_rate_pd = [str(i) for i in rate_pd]
str_pd = [str(i) for i in pd ]

In [47]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly
plotly.tools.set_credentials_file(username='gychenyolo', api_key='90gBEoe9LWXZO4iVgWj3')

In [48]:
py.sign_in('gychenyolo', '90gBEoe9LWXZO4iVgWj3')
trace1 = {
  "lat": lat, 
  "lon": lon, 
  "marker": {
    "size": str_rate_pd, 
    "sizeref": 2
  }, 
  "mode": "markers", 
  "name": "B", 
  "text": str_pd, 
  "type": "scattermapbox", 
  "uid": "469a5a"
}
data1 = Data([trace1])
layout = {
  "autosize": True, 
  "height": 511, 
  "hovermode": "closest", 
  "mapbox": {
    "bearing": 0, 
    "center": {
      "lat": 35.3116418698, 
      "lon": -118.441428557
    }, 
    "pitch": 0, 
    "zoom": 5.10790759747
  }, 
  "title": "Population density in  California", 
  "width": 910
}
fig = Figure(data=data1, layout=layout)
plot_url = py.plot(fig)

In [49]:
py.sign_in('gychenyolo', '90gBEoe9LWXZO4iVgWj3')
trace1 = {
  "lat": lat, 
  "lon": lon, 
  "marker": {
    "size": str_rate_in, 
    "sizeref": 4
  }, 
  "mode": "markers", 
  "name": "B", 
  "text": str_income, 
  "type": "scattermapbox", 
  "uid": "469a5a"
}
data1 = Data([trace1])
layout = {
  "autosize": True, 
  "height": 511, 
  "hovermode": "closest", 
  "mapbox": {
    "bearing": 0, 
    "center": {
      "lat": 35.3116418698, 
      "lon": -118.441428557
    }, 
    "pitch": 0, 
    "zoom": 5.10790759747
  }, 
  "title": "Income in  California", 
  "width": 910
}
fig = Figure(data=data1, layout=layout)
plot_url = py.plot(fig)

In [104]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_shopping_malls_in_California')
soup = BeautifulSoup(response.content, 'html.parser')
dat = soup.find_all('div', attrs = {'id':"mw-content-text", 'class':"mw-content-ltr"})
result = dat[0].find_all('li')
mall_name = []
mall_location = []
for i in result:
    try:
        name  = i.find_all('a')[0].text
        location = i.find_all('a')[1].text
        mall_name.append(name)
        mall_location.append(location)
    except:
        continue
mall_name_pd = pd.DataFrame(mall_name)
mall_locationt_pd = pd.DataFrame(mall_location)
mall_inf_pd = pd.concat([mall_locationt_pd,mall_name_pd], axis = 1)
mall_inf_pd.columns = ['Location', 'Name']

In [5]:
def art_search(url, numpages):
    '''
    Augument: 
        Given the base URl and the page number to get all the url of article.
        
    Input:
        URL and number of pages
        
    Output:
        URL
    
    '''
    urllist_1 = []
    for i in range(numpages):
        urllist = url + '&page='+ str(i+1) 
        urllist_1.append(urllist)
    return urllist_1

In [88]:
search_list = art_search('https://www.viamichelin.com/web/Restaurants?geoboundaries=23.32208,-132.4511719:43.1330612,-108.28125', 22)
zipcode = []
resaurant = []
county = []
for link in search_list:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    result = soup.find_all('div', attrs = {'class':'pois_index_list', 'data-view':'restaurants/index/list'})
    dat = result[0].find_all('li')
    for i in dat:
        try:
            inf = i.contents[2].a['href'].split('-')
            county_1 = inf[0].split('/')
            zipcode.append(inf[1])
            resaurant.append(inf[2])
            county.append(county_1[3])   
        except:
            continue
zipcode_pd = pd.DataFrame(zipcode)
resaurant_pd = pd.DataFrame(resaurant)
county_pd = pd.DataFrame(county)
resaurant_inf_pd = pd.concat([zipcode_pd, resaurant_pd, county_pd], axis = 1)
resaurant_inf_pd.columns = ['Zipcode', 'Name', 'Location']
xiao = []
for i in list(resaurant_inf_pd["Location"]):
    i = i.replace("_"," ")
    xiao.append(i)
resaurant_inf_pd["Location"] = xiao

In [103]:
def hotel_search(url, numpages):
    '''
    Augument: 
        Given the base URl and the page number to get all the url of article.
        
    Input:
        URL and number of pages
        
    Output:
        URL
    
    '''
    urllist_1 = []
    for i in range(numpages):
        urllist = url + 'pg='+ str(i+1) 
        urllist_1.append(urllist)
    return urllist_1
search_list = hotel_search('http://www.luxurylink.com/inventory/main.php?kw=california&dtin=2017-04-05&dtout=2017-04-06&guests=2&nights=1&prc=&pmn=&pmx=&es=0&ot=&sort=r&pp=25&np=1&ev=0&&', 9)
hotel_name = []
hotel_loc = []
for link in search_list:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    dat = soup.find_all('div', attrs = {"class":"large-8 columns"})
    res = dat[0].find_all('li')
    for i in res:
        try:
            location  = i.h4.small.text
            name = i.h4.contents[0]
            hotel_name.append(name)
            hotel_loc.append(location)
        except:
            continue
hotel_name_pd = pd.DataFrame(hotel_name)
hotel_loc_pd = pd.DataFrame(hotel_loc)
hotel = pd.concat([hotel_loc_pd,hotel_name_pd], axis = 1)
hotel.columns = ['Location', 'Hotel Name']

In [15]:
school_rank = []
school_location = []
school_name = []
school_name_1 = []
response = requests.get('http://www.4icu.org/us/california/universities-california.htm')
school_doc = response.text
soup = BeautifulSoup(school_doc, 'lxml')
result = soup.find_all('div', class_ = "container")
result = result[0].find_all('table',class_= "table table-hover")
result = result[0].find_all('tr')
result = result[1:]
for i in result:
    rank = i.find_all('kbd')[0].contents[0]
    school_rank.append(rank)
    name = i.find_all('td')[1].contents[0]
    school_name_1.append(name)
loc = i.find_all('td')[2].contents[0]
    school_location.append(loc)
school_name_1 = school_name_1[0:-1]
for j in school_name_1:
    school_name.append(j.contents[0])
Name = pd.DataFrame(school_name)
Rank = pd.DataFrame(school_rank)
Location = pd.DataFrame(school_location)
school = pd.concat([Rank, Name, Location], axis = 1)
school.columns = ['Rank', 'School', 'Location']
school = school.drop(school.index[-1])

In [22]:
sch_loc = list(school["Location"])

In [92]:
res_loc = list(resaurant_inf_pd["Location"])
mall_loc = list(mall_inf_pd["Location"])
hotel_loc = list(hotel["Location"])

In [59]:
def Find_loc(list_names):
    lng = []
    lat = []
    for i in list_names:
        g = geocoder.google(i+",CA")
        lat.append(g.lat)
        lng.append(g.lng)
    return lat,lng

In [68]:
import geocoder
loc1 = Find_loc(sch_loc)
lonlat1 = pd.DataFrame(loc1[0],loc1[1])
lonlat1[1] = lonlat1.index
loc2 = Find_loc(res_loc)
lonlat2 = pd.DataFrame(loc2[0],loc2[1])
lonlat2[1] = lonlat2.index
loc3 = Find_loc(mall_loc)
lonlat3 = pd.DataFrame(loc3[0],loc3[1])
lonlat3[1] = lonlat3.index

In [93]:
loc4 = Find_loc(hotel_loc)
lonlat4 = pd.DataFrame(loc4[0],loc4[1])
lonlat4[1] = lonlat4.index

In [94]:
lonlat_total = pd.concat([lonlat3,lonlat2])
lonlat_total = pd.concat([lonlat_total,lonlat4])

In [85]:
import folium
CA = (36.7783, -119.4179)
map_1 = folium.Map(CA, zoom_start=5)
map_2 = folium.Map(CA, zoom_start=5)
map_3 = folium.Map(CA, zoom_start=5)

In [98]:
marker_cluster = folium.MarkerCluster("Public toilet cluster").add_to(map_1)
for each in lonlat1.iterrows():
    folium.Marker(list((each[1][0],each[1][1]))).add_to(marker_cluster)

In [99]:
map_1

In [95]:
marker_cluster1 = folium.MarkerCluster("Public toilet cluster1").add_to(map_2)
for each in lonlat_total.iterrows():
    folium.Marker(list((each[1][0],each[1][1]))).add_to(marker_cluster1)

In [105]:
map_2